# Proyecto Final - Código de Solución
En este notebook, desarrollaremos una solución completa para predecir la tasa de cancelación de clientes de Interconnect. Seguiremos los pasos de carga de datos, preprocesamiento, entrenamiento de modelos y evaluación para encontrar el mejor modelo predictivo.

## 1. Carga y Preparación de Datos 
Comenzamos importando las librerías necesarias y cargando los cuatro archivos de datos en DataFrames de pandas. Luego, los fusionaremos en un único conjunto de datos para el análisis.

In [3]:
import pandas as pd
import numpy as np
import io
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score, accuracy_score

try:
    df_contract = pd.read_csv('contract.csv')
    df_internet = pd.read_csv('internet.csv')
    df_personal = pd.read_csv('personal.csv')
    df_phone = pd.read_csv('phone.csv')

    # Fusionar los DataFrames
    df = pd.merge(df_contract, df_personal, on='customerID', how='outer')
    df = pd.merge(df, df_internet, on='customerID', how='outer')
    df = pd.merge(df, df_phone, on='customerID', how='outer')

    print("Datos cargados y fusionados correctamente.")
    print("Forma del DataFrame final:", df.shape)
    display(df.head())

except Exception as e:
    print(f"Ocurrió un error al cargar los datos: {e}")

Datos cargados y fusionados correctamente.
Forma del DataFrame final: (7043, 20)


,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,gender,SeniorCitizen,Partner,Dependents,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,MultipleLines
0,7590-VHVEG,2020-01-01,No,Month-to-month,Yes,Electronic check,29.85,29.85,Female,0,Yes,No,DSL,No,Yes,No,No,No,No,NaN
1,5575-GNVDE,2017-04-01,No,One year,No,Mailed check,56.95,1889.5,Male,0,No,No,DSL,Yes,No,Yes,No,No,No,No
2,3668-QPYBK,2019-10-01,2019-12-01 00:00:00,Month-to-month,Yes,Mailed check,53.85,108.15,Male,0,No,No,DSL,Yes,Yes,No,No,No,No,No
3,7795-CFOCW,2016-05-01,No,One year,No,Bank transfer (automatic),42.30,1840.75,Male,0,No,No,DSL,Yes,No,Yes,Yes,No,No,NaN
4,9237-HQITU,2019-09-01,2019-11-01 00:00:00,Month-to-month,Yes,Electronic check,70.70,151.65,Female,0,No,No,Fiber optic,No,No,No,No,No,No,No


## 2. Preprocesamiento y Feature Engineering
Ahora, prepararemos los datos para el modelado. Esto incluye:
1. Crear la variable objetivo Churn.
2. Calcular la antigüedad (tenure) del cliente.
3. Limpiar y convertir tipos de datos incorrectos (TotalCharges).
4. Manejar los valores ausentes en las columnas de servicios.
5. Eliminar columnas innecesarias.

In [4]:
# 1. Crear la variable objetivo 'Churn'
df['Churn'] = (df['EndDate'] != 'No').astype(int)

# 2. Calcular la antigüedad (tenure)
# Se asume que los datos fueron extraídos el 1 de Febrero de 2020
snapshot_date = pd.to_datetime('2020-02-01')
df['BeginDate'] = pd.to_datetime(df['BeginDate'])
# Para los que cancelaron, la antigüedad es hasta la fecha de fin. Para los activos, hasta la fecha del snapshot.
df['EndDate'] = df['EndDate'].replace('No', snapshot_date.strftime('%Y-%m-%d'))
df['EndDate'] = pd.to_datetime(df['EndDate'])
df['tenure'] = ((df['EndDate'] - df['BeginDate']).dt.days / 30).round().astype(int)

# 3. Limpiar 'TotalCharges'
# Convertir a numérico, forzando errores a NaN
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
# Los NaN corresponden a clientes nuevos. Se imputarán con 0.
df['TotalCharges'].fillna(0, inplace=True)

# 4. Manejar valores ausentes en servicios
# Los NaN en estas columnas significan que el cliente no tiene el servicio correspondiente (internet o teléfono)
service_cols = ['InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
                'TechSupport', 'StreamingTV', 'StreamingMovies', 'MultipleLines']
for col in service_cols:
    df[col].fillna('No', inplace=True)

# 5. Eliminar columnas que ya no son necesarias
df.drop(['customerID', 'BeginDate', 'EndDate'], axis=1, inplace=True)

print("Preprocesamiento completado. Vista previa de los datos:")
print(df.head())
print("\nVerificación de nulos post-limpieza:")
print(df.isnull().sum().sum())

Preprocesamiento completado. Vista previa de los datos:
             Type PaperlessBilling              PaymentMethod  MonthlyCharges  \
0  Month-to-month              Yes           Electronic check           29.85   
1        One year               No               Mailed check           56.95   
2  Month-to-month              Yes               Mailed check           53.85   
3        One year               No  Bank transfer (automatic)           42.30   
4  Month-to-month              Yes           Electronic check           70.70   

   TotalCharges  gender  SeniorCitizen Partner Dependents InternetService  \
0         29.85  Female              0     Yes         No             DSL   
1       1889.50    Male              0      No         No             DSL   
2        108.15    Male              0      No         No             DSL   
3       1840.75    Male              0      No         No             DSL   
4        151.65  Female              0      No         No     Fiber opti

## 3. División de Datos y Definición de Características
Dividiremos el conjunto de datos en entrenamiento (75%) y prueba (25%). También separaremos las características numéricas de las categóricas para aplic arles el preprocesamiento adecuado.

In [5]:
# Separar características (X) y objetivo (y)
X = df.drop('Churn', axis=1)
y = df['Churn']

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

# Identificar tipos de características
categorical_features = X.select_dtypes(include=['object']).columns.tolist()
numerical_features = X.select_dtypes(include=np.number).columns.tolist()

print("División de datos completada.")
print("Características categóricas:", categorical_features)
print("Características numéricas:", numerical_features)

División de datos completada.
Características categóricas: ['Type', 'PaperlessBilling', 'PaymentMethod', 'gender', 'Partner', 'Dependents', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'MultipleLines']
Características numéricas: ['MonthlyCharges', 'TotalCharges', 'SeniorCitizen', 'tenure']


## 4. Modelo Base: Regresión Logística 
Entrenaremos un modelo simple de Regresión Logística como punto de referencia. Usaremos un Pipeline para encadenar el escalado de características numéricas y la codificación one-hot de las categóricas.

In [6]:
# Crear pipeline de preprocesamiento
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='passthrough'
)

# Crear pipeline del modelo
lr_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(random_state=42, solver='liblinear'))
])

# Entrenar el modelo
lr_pipeline.fit(X_train, y_train)

# Realizar predicciones
y_pred_lr = lr_pipeline.predict(X_test)
y_proba_lr = lr_pipeline.predict_proba(X_test)[:, 1]

# Evaluar
auc_lr = roc_auc_score(y_test, y_proba_lr)
acc_lr = accuracy_score(y_test, y_pred_lr)

print("--- Regresión Logística ---")
print(f"AUC-ROC: {auc_lr:.4f}")
print(f"Accuracy: {acc_lr:.4f}")

--- Regresión Logística ---
AUC-ROC: 0.8463
Accuracy: 0.8069


## 5. Modelo Avanzado: CatBoost 
Ahora entrenaremos un modelo más sofisticado, CatBoost, que es conocido por su buen rendimiento y su manejo nativo de variables categóricas.

In [7]:
# Inicializar y entrenar el modelo CatBoost
# Le pasamos directamente las características categóricas
cat_model = CatBoostClassifier(
    iterations=500,
    learning_rate=0.1,
    depth=6,
    cat_features=categorical_features,
    random_state=42,
    verbose=0,  # Suprimir el output durante el entrenamiento
    eval_metric='AUC'
)

cat_model.fit(X_train, y_train)

# Realizar predicciones
y_pred_cat = cat_model.predict(X_test)
y_proba_cat = cat_model.predict_proba(X_test)[:, 1]

# Evaluar
auc_cat = roc_auc_score(y_test, y_proba_cat)
acc_cat = accuracy_score(y_test, y_pred_cat)

print("--- CatBoost Classifier ---")
print(f"AUC-ROC: {auc_cat:.4f}")
print(f"Accuracy: {acc_cat:.4f}")

--- CatBoost Classifier ---
AUC-ROC: 0.8374
Accuracy: 0.7922


In [9]:
from sklearn.model_selection import GridSearchCV

# 1. Definir el espacio de búsqueda de parámetros
# (Estos son rangos de ejemplo, se pueden ampliar)
param_grid = {
    'iterations': [200, 500, 700],
    'learning_rate': [0.03, 0.1],
    'depth': [4, 6, 8],
    'l2_leaf_reg': [1, 3, 5]
}

# 2. Inicializar el modelo base de CatBoost (sin entrenar)
cat_model_grid = CatBoostClassifier(
    cat_features=categorical_features,
    random_state=42,
    verbose=0,
    eval_metric='AUC'
)

# 3. Configurar GridSearchCV
# cv=3 significa validación cruzada de 3 pliegues.
# n_jobs=-1 usa todos los procesadores disponibles para acelerar la búsqueda.
grid_search = GridSearchCV(
    estimator=cat_model_grid,
    param_grid=param_grid,
    scoring='roc_auc',
    cv=3,
    n_jobs=-1
)

# 4. Ejecutar la búsqueda (esto puede tardar unos minutos)
print("Iniciando la búsqueda de hiperparámetros...")
grid_search.fit(X_train, y_train)

# 5. Obtener los mejores parámetros y el mejor score
print(f"Mejor AUC-ROC en validación cruzada: {grid_search.best_score_:.4f}")
print("Mejores hiperparámetros encontrados:")
print(grid_search.best_params_)

# 6. Evaluar el mejor modelo encontrado en el conjunto de prueba
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test)
y_proba_best = best_model.predict_proba(X_test)[:, 1]

auc_best = roc_auc_score(y_test, y_proba_best)
acc_best = accuracy_score(y_test, y_pred_best)

print("\n--- Resultados del Modelo CatBoost Optimizado ---")
print(f"AUC-ROC en el conjunto de prueba: {auc_best:.4f}")
print(f"Accuracy en el conjunto de prueba: {acc_best:.4f}")

Iniciando la búsqueda de hiperparámetros...
Mejor AUC-ROC en validación cruzada: 0.8482
Mejores hiperparámetros encontrados:
{'depth': 4, 'iterations': 500, 'l2_leaf_reg': 3, 'learning_rate': 0.03}

--- Resultados del Modelo CatBoost Optimizado ---
AUC-ROC en el conjunto de prueba: 0.8489
Accuracy en el conjunto de prueba: 0.8007


## 6. Conclusión de la Solución en Código 
Hemos desarrollado, entrenado y evaluado dos modelos. A continuación se comparan sus resultados:

In [11]:
# Crear un DataFrame para comparar los resultados de los modelos
results_df = pd.DataFrame({
    'Modelo': ['Regresión Logística', 'CatBoost Classifier','CatBoost Optimizado'],
    'AUC-ROC': [auc_lr, auc_cat, auc_best],
    'Accuracy': [acc_lr, acc_cat, acc_best]
})

# Establecer 'Modelo' como el índice para una mejor visualización
results_df = results_df.set_index('Modelo')

# Ordenar los resultados por la métrica principal (AUC-ROC) de forma descendente
results_df = results_df.sort_values(by='AUC-ROC', ascending=False)

print("--- Tabla Comparativa de Resultados ---")
print(results_df)

--- Tabla Comparativa de Resultados ---
                      AUC-ROC  Accuracy
Modelo                                 
CatBoost Optimizado  0.848926  0.800681
Regresión Logística  0.846311  0.806928
CatBoost Classifier  0.837389  0.792164


El CatBoost Optimizado es el modelo superior porque ofrece el mejor rendimiento en la métrica principal del proyecto. Con un AUC-ROC de 0.8489, se sitúa en la categoría de 4.5 SP (0.81 ≤ AUC-ROC < 0.85), demostrando una buena capacidad para predecir la cancelación de clientes.

Cabe mencionar que aunque la Regresión Logística tiene una exactitud (Accuracy) ligeramente superior, la diferencia es muy pequeña. Dado que AUC-ROC es la métrica prioritaria, la ligera ventaja en exactitud de la regresión logística no es suficiente para que sea elegida como el mejor modelo.